# Data preprocessing - Creation of the final data frame for data analysis

## Introduction

This document contains the code for the item creation corresponding to the master thesis "...". 

The research question of the thesis is: Does the similarity of the first three noun phrase antecedents of double embedded sentences have an effect on the reading times of grammatical and ungrammatical double embedded sentences?

To examine this question the self-paced-reading (SPR) and eye-tracking data of Vasishth et al. (2010) are examined. The similarity of the three noun phrase antecedents is calculated with the machine learning model Word2vec for each sentence. A linear mixed model with varying intercepts for subjects is fitted that measures the effect of the interaction between the similarity and the grammaticality of the sentences.
In the following paragraphs the exact procedure for the creation of items as well as the actual code of item creation is presented. 

End product of this notebook: A data frame that contains the subject ID, the sentence number, the condition of the sentence, the reading time of the post V1 region (log transformed) and the corresponding similarity value (centred). The data was empirically collected for the Vasishth et al. (2010) study.

## Coding part

At first, the required packages have to be downloaded. Pandas and numpy are both libraries that provide functions for Python, enabling the user to deal with data or mathematical structures.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # ist das nötig?

Next, the data corresponding to the fifth English SPR experiment of the Vasishth et al. (2010) study is uploaded into the notebook.

In [7]:
df = pd.read_csv('e5_en_spr_data.txt', sep=' ', names=['Subject_ID', 'Experiment', 'Sentence_Number', 'Condition', 'Word_Position','Word', '-','RT'])
df

,Subject_ID,Experiment,Sentence_Number,Condition,Word_Position,Word,-,RT
0,1,filler,67,practice,0,The,-,418
1,1,filler,67,practice,1,ski,-,448
2,1,filler,67,practice,2,instructor,-,506
3,1,filler,67,practice,3,warned,-,579
4,1,filler,67,practice,4,the,-,468
...,...,...,...,...,...,...,...,...
59189,999,E1,17,a,10,plan,-,225
59190,999,E1,17,a,11,was,-,207
59191,999,E1,17,a,12,extremely,-,215
59192,999,E1,17,a,13,inconvenient.,-,272


### Extraction of relevant data 

A lot of data points are not needed for the purpose of this study. Only the post V1 region of the 'gug' condition (the experimental condition) is needed. The relevant data has to be extracted of the data frame.

In [8]:
# Extract the experimental condition 'gug'
df_gug = df[df.Experiment=='gug']
df_gug

,Subject_ID,Experiment,Sentence_Number,Condition,Word_Position,Word,-,RT
152,1,gug,13,a,0,The,-,291
153,1,gug,13,a,1,"conductor,",-,592
154,1,gug,13,a,2,who,-,378
155,1,gug,13,a,3,the,-,356
156,1,gug,13,a,4,"choirmaster,",-,2747
...,...,...,...,...,...,...,...,...
59072,50,gug,12,a,9,"disturbed,",-,1024
59073,50,gug,12,a,10,visited,-,504
59074,50,gug,12,a,11,the,-,1432
59075,50,gug,12,a,12,grandfather.,-,657


All words of the "gug" condition are now extracted and stored in a new data frame called df_gug. 
Since only the post V1 region is relevant for the data analysis the relevant rows, corresponding to the post V1 region, have to be extracted from the data frame df_gug. The post V1 region is always the second last word of every sentence.

How to get the second last word of every sentence:
Since all sentences differ in their length, it is not directly possible to access the second last word of each sentence. We therefore try first to find out the word positions of the last words of all sentences to be able to extract the second last words.

Our procedure to find the last word of every sentence works as follows: We start by finding the words that occur with a dot since every sentence ends with a dot. Therefore, the most straight forward way to find out where sentences end is to find out the positions of these dots. To do so, we create a Boolean series that gives us a TRUE when a dot occurs after a word and a FALSE when this is not the case.

In [9]:
last_word = df_gug.Word.apply(lambda x: '.' in x)
last_word

152      False
153      False
154      False
155      False
156      False
         ...  
59072    False
59073    False
59074    False
59075     True
59076    False
Name: Word, Length: 11125, dtype: bool

Next, we create an array 'index_last_word' that extracts the words' numbers that have a TRUE as output in our Boolean series (thus, that occur with a dot in the end). As a consequence, we have the indices of all last words of all sentences.

In [10]:
index_last_word = np.where(last_word)[0]

Since we don't need the last word but the second last word, we subtract 1 from each index and store the numbers in an array index_second_last. We then extract all words that correspond to the numbers in the array index_second_last.

In [11]:
# index of second last word
index_second_last = index_last_word -1

In [12]:
second_last_word = df_gug.iloc[index_second_last]
second_last_word

,Subject_ID,Experiment,Sentence_Number,Condition,Word_Position,Word,-,RT
163,1,gug,13,a,11,the,-,612
176,1,gug,12,d,10,the,-,761
189,1,gug,15,c,10,the,-,347
241,1,gug,1,a,11,the,-,490
321,1,gug,5,a,12,the,-,290
...,...,...,...,...,...,...,...,...
58967,50,gug,8,a,11,the,-,408
58999,50,gug,5,b,12,the,-,1152
59035,50,gug,13,b,11,the,-,360
59048,50,gug,15,d,10,the,-,650


### The similarity values

In this part we first import the nouns with two conditions, the similar and the contrast condition, and their corresponding similarity values. Next the concept of centred similarity is explained and carried out.

We now import the similarity values and store them into a data frame called Simval_df. 
We have two similarity values for each sentence. One similarity value is for the similar condition with three animate noun phrases, one for the contrast condition with two animate nouns and one inanimate noun in the middle. 
The similarity values are the mean similarity of the three nouns respectively.

In [14]:
Simval_df = pd.read_csv('df_SimilarityValues.csv', sep=',', names=['Sentence_Number', 'Similar_condition', 'Contrast_condition', 'SimVal_Similar', 'SimVal_Contrast'])
Simval_df

,Sentence_Number,Similar_condition,Contrast_condition,SimVal_Similar,SimVal_Contrast
0,0,"('carpenter', 'craftsman', 'peasant')","('carpenter', 'pillar', 'peasant')",0.391551,0.137571
1,1,"('mother', 'daughter', 'sister')","('mother', 'gun', 'sister')",0.810291,-0.069852
2,2,"('worker', 'tenant', 'foreman')","('worker', 'bucket', 'foreman')",0.369354,0.225532
3,3,"('trader', 'businessman', 'professor')","('trader', 'computer', 'professor')",0.623997,0.101565
4,4,"('painter', 'musician', 'father')","('painter', 'hut', 'father')",0.423884,0.271721
5,5,"('saxophonist', 'trumpeter', 'conductor')","('saxophonist', 'baton', 'conductor')",0.703727,0.308906
6,6,"('pharmacist', 'optician', 'stranger')","('pharmacist', 'button', 'stranger')",0.295159,0.233564
7,7,"('cleaner', 'janitor', 'doctor')","('cleaner', 'ball', 'doctor')",0.315141,0.207383
8,8,"('dancer', 'singer', 'bystander')","('dancer', 'shoe', 'bystander')",0.568988,0.452954
9,9,"('artist', 'sportsman', 'guard')","('artist', 'computer', 'guard')",0.253536,0.001793


The concept of centred similarity is an equivalent to the sum contrast coding that will be used for the data analysis later on. 
In the sum contrast coding each data point is coded as a function of the overall mean of all data points.
For the sum contrast coding for the condition of grammaticality we have +1 for grammatical and -1 for ungrammatical sentences.
For the examination of the research question we have to carry out an interaction analysis to see the impact of the similarity values (hence the condition of similarity) on the condition of grammaticality. 
For this purpose, the similarity values have to be adapted to the sum contrast coding concept. This is done by subtracting the mean similarity value from each individual similarity value.

First we calculate the mean similarity value from the similar and the contrast condition.

In [15]:
Simval_Similar = list(Simval_df['SimVal_Similar'])
Simval_Contrast = list(Simval_df['SimVal_Contrast'])

In [16]:
added_sim = Simval_Similar + Simval_Contrast

In [17]:
mean_sim = np.mean(added_sim)
mean_sim

0.3123188643658068

Next, we need to subtract the mean from each similarity value.

This is done with a for loop that loops over the Simval_Similar list (containing all similarity values corresponding to the noun pairs of the similar group) or Simval_Contrast list (containing all similarity values corresponding to the noun pairs of the contrast group). For each value of the list, we subtract the mean from that value and store the result in a list called Centred_simval_similar or Centred_simval_contrast.

In [18]:
Centred_simval_similar = []
for value in Simval_Similar:
    Centred_simval_similar.append(value - mean_sim)
print(Centred_simval_similar)

[0.0792321980989073, 0.497972247103462, 0.05703535387874581, 0.3116784066951368, 0.1115649611747358, 0.39140784452320077, -0.017159967770567164, 0.0028219790256117094, 0.2566690714156721, -0.0587831987941172, -0.08934552324353714, 0.38843387077213265, -0.039501111110439524, 0.14478000710369088, 0.20878489772439934, 0.23193129608989693]


In [19]:
Centred_simval_contrast = []
for value in Simval_Contrast:
    Centred_simval_contrast.append(value - mean_sim)
print(Centred_simval_contrast)

[-0.1747479225450661, -0.38217081417678855, -0.0867871894442942, -0.21075361417024396, -0.04059784402488731, -0.003413009544601664, -0.07875486006378196, -0.10493575324653648, 0.140635010699043, -0.3105253755056765, -0.23551733707427047, -0.2871846380585339, -0.007859083911171183, -0.2122081099951174, -0.36469443430542015, -0.11800735731958412]


For our end data frame we need log transformed reading times.

In [20]:
logRT = list(np.log(second_last_word['RT']))

The next section is about merging the correct similarity value to its corresponding sentence number,  condition and reading time. 

To do that, we first create a data frame containing all the relevant information for our end data frame, except of the similarity values and the contrast coding for the condition of grammaticality (+1 and -1).

In [21]:
df_WithoutSim = pd.DataFrame(list(zip(second_last_word.Subject_ID, second_last_word.Sentence_Number, second_last_word.Condition, logRT)),
                            columns = ['Subject_ID', 'Sentence_Number', 'Condition', 'logRT'])
df_WithoutSim

,Subject_ID,Sentence_Number,Condition,logRT
0,1,13,a,6.416732
1,1,12,d,6.634633
2,1,15,c,5.849325
3,1,1,a,6.194405
4,1,5,a,5.669881
...,...,...,...,...
795,50,8,a,6.011267
796,50,5,b,7.049255
797,50,13,b,5.886104
798,50,15,d,6.476972


The data frame we need for the statistical analysis would have one more column containing the centred similarity values. We have two similarity values for all sentences (1-16). Condition a and c need the similarity values of the similar condition, condition b and d need the similarity value of the contrast condition.

To do that, we write a for loop that loops over the list “Sentence_Number” of our data frame “df_WithoutSim”. Note that the list “Sentence_Number” does not only contain the numbers from 1 to 16! It has length 800 and contains the sentence numbers that correspond to the sentences that have been read by a specific participant. The ‘iterater’ variable starts to count from zero until the end of a list. In other words, it assigns an index, stating at zero until the end, for every time we move on to the next item of our for loop. The variable ‘i’ refers to the entry at a specific point in the list “Sentence_Number”. The function enumerate makes the for loop loop through the whole list “Sentence_Number”.  Assume we start at the first entry in Sentence_Number. The first entry is the sentence 14, read by participant 1. Thus, our iterater would start counting at zero and the variable ‘i’ would get 14. Before jumping to the next entry, we now want to know which condition this entry has. This is important in order to assign the correct similarity value to that entry. Therefore, we look at the entry in the “Condition” column of our data frame “df_WithoutSim” in the same row (in our example row zero). If the entry is either ‘a’ or ‘c’ (similar condition) we append the centred similarity value for the similar group of that same sentence in an empty list called “Similarity_Values_dat”. In our example we would append the centred similarity value for the contrast condition of sentence 14 ~-0.212, since the condition in the first entry of the data frame is 'b'. Thereby, we have to take the entry i-1 of the “Centred_simval_similar” list. We have to take i-1 since Python starts counting at zero. ‘i’ is the correct sentence number. In our example i=14. In the “Centred_simval_similar” list we have all the similarity values for all sentences in the correct order, starting from zero to fifteen. If we want to add the similarity value of the fourteenth sentence, we have to take the thirteenth entry of the “Centred_simval_similar” list. If the condition is not ‘a’ or ‘c’ (meaning it has to be ‘b’ or ‘d’) we want to add the corresponding contrast centred similarity value. We then take the i-1th entry of the “Centered_simval_contrast” list and append it to the “Similarity_Values_dat” list. After that, enumerate makes us loop to the next entry of “Sentence_Number” etc.

In [22]:
Similarity_Values_dat = []

for iterater, i in enumerate(df_WithoutSim.Sentence_Number):
    if df_WithoutSim.Condition[iterater] in ['a', 'c']:
        simVal_similar = Centred_simval_similar[i-1]
        Similarity_Values_dat.append(simVal_similar)
    else:
        simVal_Con = Centred_simval_contrast[i-1]
        Similarity_Values_dat.append(simVal_Con)
print(Similarity_Values_dat)

[-0.039501111110439524, -0.2871846380585339, 0.20878489772439934, 0.0792321980989073, 0.1115649611747358, -0.2122081099951174, 0.05703535387874581, -0.003413009544601664, 0.2566690714156721, -0.017159967770567164, -0.10493575324653648, -0.38217081417678855, -0.3105253755056765, -0.21075361417024396, -0.08934552324353714, -0.11800735731958412, 0.140635010699043, -0.1747479225450661, -0.36469443430542015, 0.3116784066951368, 0.23193129608989693, 0.0028219790256117094, -0.04059784402488731, 0.497972247103462, -0.07875486006378196, 0.38843387077213265, 0.14478000710369088, -0.0587831987941172, -0.007859083911171183, -0.23551733707427047, -0.0867871894442942, 0.39140784452320077, -0.38217081417678855, -0.3105253755056765, 0.05703535387874581, -0.017159967770567164, -0.10493575324653648, -0.2122081099951174, 0.1115649611747358, -0.2871846380585339, 0.0792321980989073, -0.003413009544601664, -0.21075361417024396, -0.11800735731958412, 0.2566690714156721, 0.20878489772439934, -0.08934552324353

For the final data frame for data analysis we need the grammaticality condition to be coded as + and - 1. Conditions a and b refer to grammatical sentences, condition c and d to ungrammatical sentences. We iterate over the column "Condition" of our data frame 'df_WithoutSim'. If the condition is equal to 'a' or 'b' we enter a 1 in the list 'Contrast_Coding', else (if condition is equal to 'c' or 'd') we enter -1 in the list. This list can later be added in the data frame in a new column called "Contrast_coding".

In [23]:
Contrast_Coding = []

for cond in df_WithoutSim.Condition:
    if cond in ['a','b']:
        Contrast_Coding.append(1)
    else:
        Contrast_Coding.append(-1)
print(Contrast_Coding)

[1, -1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1,

Now we have everything we need to create the final data frame, thus to add a column for the contrast coding numbers and the centred similarity values.

In [24]:
df_WithSim = pd.DataFrame(list(zip(df_WithoutSim.Subject_ID, df_WithoutSim.Sentence_Number, df_WithoutSim.Condition, Contrast_Coding, logRT, Similarity_Values_dat)),
                            columns = ['Subject_ID', 'Sentence_Number', 'Condition', 'Contrast_Coding','logRT', 'Centered_Sim_Val'])
df_WithSim

,Subject_ID,Sentence_Number,Condition,Contrast_Coding,logRT,Centered_Sim_Val
0,1,13,a,1,6.416732,-0.039501
1,1,12,d,-1,6.634633,-0.287185
2,1,15,c,-1,5.849325,0.208785
3,1,1,a,1,6.194405,0.079232
4,1,5,a,1,5.669881,0.111565
...,...,...,...,...,...,...
795,50,8,a,1,6.011267,0.002822
796,50,5,b,1,7.049255,-0.040598
797,50,13,b,1,5.886104,-0.007859
798,50,15,d,-1,6.476972,-0.364694


In [25]:
# We are now ready to export the data frame for the data analysis in r
df_WithSim.to_csv('Final_DataFrame_e5.csv', sep=',', header=False, index=True)